In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
from keras.optimizers import Adam
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/livelab/Desktop/NewResizedBinned'
checkAndGeneratePath(parentPath)
testPath='/home/livelab/Desktop/NewResizedBinned/Test'
trainPath='/home/livelab/Desktop/NewResizedBinned/Training'
validPath='/home/livelab/Desktop/NewResizedBinned/Validation'

trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

Length Check on Folder Succeed


In [ ]:
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'
combo=prepdata(trainPath)
len(combo)
a,b=zip(*combo)
b=np.argmax(a,axis=1)
b=b.tolist()
for i in range(24):
    print(b.count(i),i)

In [3]:
def loadCNN(weightFile):
    """
    Load pretrained CNN and lock all features
    """
    #full inception
#freeze mix3 and prior
    inputData=Input(shape=(256, 256, 3))
    model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 256, 256, 3))
    intermediateOut = Flatten()(model.output)
    intermediateOut = Dense(32)(intermediateOut)
    intermediateOut = BatchNormalization()(intermediateOut)
    intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
    intermediateOut = Dropout(0.4)(intermediateOut)
    prediction = Dense(24, activation='softmax')(intermediateOut)
    optmr = Adam(lr=0.0001, decay = 5e-4)
    model_final = Model(input = model.input, output = prediction)
    model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])
    model_final.load_weights(weightFile)
    #retrieve output of second to last layer
    lenOfModel=len(model_final.layers)
    intermediate_layer_model = Model(inputs=model_final.input,
                               outputs=model_final.layers[lenOfModel-2].output)
    for layer in intermediate_layer_model.layers: #until mixed9_0, untrainable
        layer.trainable = False
    return intermediate_layer_model

In [4]:
def findLayerNumByName(model, name):
    layerList=model.layers
    return layerList.index(model.get_layer(layer_name))

In [8]:
model=loadCNN('/home/livelab/Desktop/CELA-VideoClassification/CNN/model-full-10k-0.54.hdf5')

In [32]:
#Using TimeDistributed Wrapper
inputData=Input(shape=(None, 256, 256, 3))
model=loadCNN('/home/livelab/Desktop/CELA-VideoClassification/CNN/model-full-10k-0.54.hdf5')
encoded_frame = TimeDistributed(model)(inputData)

intermediateOut = CuDNNLSTM(1024, return_sequences = False)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = Dense(128)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
# intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(24, activation='softmax')(intermediateOut)
optmr = keras.optimizers.adam(lr=0.005, decay = 5e-4)
model_final = Model(input = [inputData], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])
filepath = "CRNN-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

In [33]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 32)          24162240  
_________________________________________________________________
cu_dnnlstm_7 (CuDNNLSTM)     (None, 1024)              4333568   
_________________________________________________________________
batch_normalization_1051 (Ba (None, 1024)              4096      
_________________________________________________________________
dense_30 (Dense)             (None, 24)                24600     
Total params: 28,524,504
Trainable params: 4,360,216
Non-trainable params: 24,164,288
_________________________________________________________________


In [34]:
STEP_SIZE_TRAIN=trainingGen.__len__()//trainingGen.batch_size/5
STEP_SIZE_VAL=validGen.__len__()//validGen.batch_size

In [ ]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validGen,
                    validation_steps=STEP_SIZE_VAL,
                    callbacks=callbacks_list,
                    use_multiprocessing=True,
                    epochs = 40,
                    workers=6)

Epoch 1/40
1678/1678 [==============================] - 266s 159ms/step - loss: 2.9797 - acc: 0.1544 - val_loss: 11.1260 - val_acc: 0.0775

Epoch 00001: val_acc improved from -inf to 0.07746, saving model to CRNN-01-0.08.hdf5
Epoch 2/40
1678/1678 [==============================] - 217s 129ms/step - loss: 2.9321 - acc: 0.1633 - val_loss: 10.4367 - val_acc: 0.0467

Epoch 00002: val_acc did not improve from 0.07746
Epoch 3/40
1678/1678 [==============================] - 219s 131ms/step - loss: 2.9374 - acc: 0.1704 - val_loss: 11.1295 - val_acc: 0.0209

Epoch 00003: val_acc did not improve from 0.07746
Epoch 4/40
1677/1678 [============================>.] - ETA: 0s - loss: 2.9591 - acc: 0.1676

In [ ]:
model_json = model_final.to_json()
with open("modelmix3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("modelmix3.h5")

In [ ]:
json_file = open('saved-model-07-0.16.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("saved-model-07-0.16.h5")
print("Loaded model from disk")

In [ ]:
model_final.load_weights("saved-model-2-01-0.17.hdf5")

In [ ]:
#manually test model using testset
testGen=imgDataGenerator(testPath)
correct=0
laxcorrect=0
with tf.device('GPU:0'):
    for i in range(193):
        x,y=testGen.__getitem__(i)
        pred=model_final.predict(x)
        pred=pred.reshape((len(pred[0]),24))
        spred=np.sum(pred,axis=0)
        aspred=np.argsort(-spred)
#         print(aspred[:3])
#         print(aspred)
#         print(np.argmax(spred))
#         print(spred)
#         print(np.argmax(y[0][0]))
#         print('**')
        if(np.argmax(y[0][0]) in aspred[:1]):
            correct+=1
        if(np.argmax(y[0][0]) in aspred[:3]):
            laxcorrect+=1    
print('Accuracy of Testing Set: ',correct/193, laxcorrect/193)

In [7]:
#manually test model using testset
testGen=imgDataGenerator(testPath)
correct=0
laxcorrect=0
with tf.device('GPU:0'):
    x,y=testGen.__getitem__(0)
    pred=model_final.predict(x)
    pred=pred.reshape((len(pred[0]),24))
    print(pred[0])
    for p in pred:
        print(np.argmax(p))
    print(np.argmax(y))
    print(y)
print('Accuracy of Testing Set: ',correct/193, laxcorrect/193)

[2.4444167e-02 5.1272457e-04 3.3864358e-03 1.3929498e-02 8.7655790e-04
 3.9349613e-04 2.0848449e-04 2.9556248e-03 2.6248468e-03 2.7816342e-02
 2.7376935e-03 1.1535666e-02 1.9808723e-04 3.3293080e-03 3.6888491e-02
 7.2536373e-01 7.9956785e-04 1.6302963e-03 4.3726549e-03 1.0733723e-03
 3.4537264e-03 1.2404115e-01 6.8205590e-03 6.0759677e-04]
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
14
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Accuracy of Testing Set:  0.0 0.0


## testGen=imgDataGenerator(testPath)
combo=prepdata(testPath)

In [ ]:
a,b=zip(*combo)

In [ ]:
len(a)

In [ ]:
a=[1,2,3,4,5]
np.random.shuffle(a)

In [ ]:
a